# Polar Coding Notebook

Helper Functions and Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import log2

def is_power_of_two(n):
    return (n & (n-1)) == 0 and n > 0


def bit_reverse_indices(N):
    n = int(np.log2(N))
    br = np.zeros(N, dtype=int)
    for i in range(N):
        br[i] = int(format(i, 'b').zfill(n)[::-1], 2)
    return br


def construct_G_nat(N):
    assert is_power_of_two(N), "N must be power of two"
    F = np.array([[1,0],[1,1]], dtype=int)
    n = int(np.log2(N))
    G = np.array([[1]], dtype=int)
    for _ in range(n):
        G = np.kron(G, F)
    return G % 2

Channels

In [2]:
def bsc(x, p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return (x ^ flips).astype(np.int8)


def bsc_llr(y, p, eps=1e-12):
    p_safe = min(max(float(p), eps), 1.0 - eps)
    alpha = np.log((1.0 - p_safe) / p_safe)
    return (1 - 2*y) * alpha

LLR (Log-Likelihood Ratios) Combines

In [3]:
def f_min_sum(a, b):
    a = np.array(a); b = np.array(b)
    return np.sign(a * b) * np.minimum(np.abs(a), np.abs(b))


def f_exact(a, b, eps=1e-12):
    A = np.array(a, dtype=float)
    B = np.array(b, dtype=float)
    ta = np.tanh(A / 2.0)
    tb = np.tanh(B / 2.0)
    t = ta * tb
    t = np.clip(t, -1.0 + eps, 1.0 - eps)
    return np.log1p(t) - np.log1p(-t)

Polar Coding Functions

In [4]:
def encode_with_G_nat(u):
    N = len(u)
    G = construct_G_nat(N)
    x = (u.astype(int) @ G.astype(int)) % 2
    return x.astype(np.int8)


def polar_encode_with_G(msg, info_positions, N, u_frozen=None):
    if u_frozen is None:
        u_frozen = np.zeros(N, dtype=np.int8)
    u = np.array(u_frozen, dtype=np.int8)
    u[np.sort(info_positions)] = np.array(msg, dtype=np.int8)
    x = encode_with_G_nat(u)
    return x.astype(np.int8), u.astype(np.int8)


def bhattacharyya_parameter(p, N):
    Z = np.array([2.0 * np.sqrt(p * (1.0 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append(2*z - z*z)  # Upper
            Z_next.append(z*z) # Lower
        Z = np.array(Z_next)
    return Z


def select_frozen_direct(N, K, p_design):
    Z = bhattacharyya_parameter(p_design, N)
    logical_best = np.argsort(Z)[:K]
    info_positions_natural = np.sort(logical_best)
    frozen_mask = np.ones(N, dtype=bool)
    frozen_mask[info_positions_natural] = False
    return frozen_mask, info_positions_natural


def sc_decode(llr, frozen_mask, u_frozen=None, use_exact_f=False):
    if u_frozen is None:
        u_frozen = np.zeros(len(frozen_mask), dtype=np.int8)
    llr = np.array(llr, dtype=float)
    frozen_mask = np.array(frozen_mask, dtype=bool)
    u_frozen = np.array(u_frozen, dtype=np.int8)

    def rec(llr_sub, frozen_sub, u_frozen_sub):
        n = len(llr_sub)
        if n == 1:
            if frozen_sub[0]:
                return np.array([u_frozen_sub[0]], dtype=np.int8)
            else:
                return np.array([0 if llr_sub[0] >= 0 else 1], dtype=np.int8)
        n2 = n // 2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f = f_exact(a,b) if use_exact_f else f_min_sum(a,b)
        uhat_up = rec(f, frozen_sub[:n2], u_frozen_sub[:n2])
        sign_term = (1 - 2 * uhat_up)
        g = b + sign_term * a
        uhat_low = rec(g, frozen_sub[n2:], u_frozen_sub[n2:])
        u_upper = uhat_up.astype(np.int8)
        u_lower = (uhat_up ^ uhat_low).astype(np.int8)
        return np.concatenate([u_upper, u_lower]).astype(np.int8)

    return rec(llr, frozen_mask, u_frozen)

Tests

In [5]:
def instrumented_n4_matrix(p_design=0.05):
    N = 128; K = 2
    frozen_mask, info_positions = select_frozen_direct(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    msg = np.array([0,1], dtype=np.int8)
    x, u = polar_encode_with_G(msg, info_positions, N, u_frozen=u_frozen)
    print("info_positions (natural):", info_positions)
    print("u (natural):", u.tolist())
    print("x:", x.tolist())
    y = x.copy()
    llr = bsc_llr(y, 1e-12)
    print("llr:", llr.tolist())
    # instrument SC recursion
    def sc_print(llr_sub, frozen_sub, u_frozen_sub, depth=0, path="root"):
        indent = "  "*depth
        n = len(llr_sub)
        print(f"{indent}{path}: n={n}, llr_sub={llr_sub.tolist()}, frozen_sub={frozen_sub.tolist()}")
        if n==1:
            if frozen_sub[0]:
                print(f"{indent} => leaf frozen -> {u_frozen_sub[0]}")
                return np.array([u_frozen_sub[0]], dtype=np.int8)
            dec = 0 if llr_sub[0] >= 0 else 1
            print(f"{indent} => leaf decide -> {dec}")
            return np.array([dec], dtype=np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f_llr = f_exact(a,b)
        print(f"{indent} f: a={a.tolist()}, b={b.tolist()} -> f={f_llr.tolist()}")
        uhat_up = sc_print(f_llr, frozen_sub[:n2], u_frozen_sub[:n2], depth+1, path=path+"-L")
        print(f"{indent} uhat_up returned: {uhat_up.tolist()}")
        sign_term = (1 - 2 * uhat_up)
        g_llr = b + sign_term * a
        print(f"{indent} g: sign_term={sign_term.tolist()}, g_llr={g_llr.tolist()}")
        uhat_low = sc_print(g_llr, frozen_sub[n2:], u_frozen_sub[n2:], depth+1, path=path+"-R")
        print(f"{indent} uhat_low returned: {uhat_low.tolist()}")
        combined = np.concatenate([uhat_up.astype(np.int8), (uhat_up ^ uhat_low).astype(np.int8)]).astype(np.int8)
        print(f"{indent} combined: {combined.tolist()}")
        return combined
    uhat = sc_print(llr, frozen_mask, u_frozen)
    print("decoded uhat:", uhat.tolist())
    print("expected u:", u.tolist())
    print("match?", np.array_equal(uhat, u))
    return uhat, u, frozen_mask, info_positions


In [6]:
uhat, u_expected, frozen_mask_ex, info_pos = instrumented_n4_matrix()

info_positions (natural): [126 127]
u (natural): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111

GPT Tests

In [7]:
def build_G_matrix(N):
    """
    Build natural-order polar generator matrix (Arikan) of size N×N.

    Recursion:
       G_1 = [1]
       G_{2N} = [[G_N, G_N],
                [0_N, G_N]]
    """
    import numpy as np
    if N == 1:
        return np.array([[1]], dtype=np.int8)

    G_half = build_G_matrix(N//2)

    top = np.concatenate((G_half, G_half), axis=1)
    bottom = np.concatenate((np.zeros_like(G_half, dtype=np.int8), G_half), axis=1)

    return np.concatenate((top, bottom), axis=0).astype(np.int8)


# build global G_matrix
N=8
G_matrix = build_G_matrix(N)

In [8]:
def deterministic_debug(N=8, K=4, p_design=0.05):
    print("=== Deterministic Debug ===")

    frozen_mask, info_positions = select_frozen_direct(N, K, p_design)
    info_positions = np.sort(info_positions)
    print("info_positions:", info_positions)
    print("frozen_mask:", frozen_mask.astype(int))

    # build generator matrix
    G = build_G_matrix(N)

    # create message
    msg = np.random.randint(0,2,size=len(info_positions)).astype(np.int8)
    print("msg:", msg)

    # build full u
    u = np.zeros(N, dtype=np.int8)
    u[info_positions] = msg

    print("u (full):", u)

    # encode
    x = (G @ u) % 2
    print("x:", x)

    # llr (no noise)
    # artificially: llr = VERY LARGE positive for zeros, negative for ones
    llr = (1 - 2*x) * 1000.0
    print("llr:", llr)

    # --- map frozen_mask from natural -> decoder (bit-reversed) order ---
    br = bit_reverse_indices(N)        # maps logical->natural
    inv_br = np.empty_like(br); inv_br[br] = np.arange(N)   # maps natural->logical (decoder order)

    frozen_mask_br = frozen_mask[inv_br]      # reorder mask for decoder
    u_frozen_nat = np.zeros(N, dtype=np.int8)
    u_frozen_br = u_frozen_nat[inv_br]

    # decode using decoder-order frozen mask
    uhat_br = sc_decode(
        llr,            # sc_decode expects LLR in decoder tree natural-order (we're using natural-order llr but sc traversal expects the same ordering convention)
        frozen_mask_br,
        use_exact_f=True
    )

    # map decoded back to natural order
    # uhat_br is in decoder/logical order; map back:
    uhat_nat = uhat_br[inv_br]
    print("u_hat (natural order):", uhat_nat)
    print("expected info:", u[info_positions])
    print("decoded info :", uhat_nat[info_positions])
    print("match?", np.array_equal(u[info_positions], uhat_nat[info_positions]))


deterministic_debug()


=== Deterministic Debug ===
info_positions: [3 5 6 7]
frozen_mask: [1 1 1 0 1 0 0 0]
msg: [1 1 0 1]
u (full): [0 0 0 1 0 1 0 1]
x: [1 1 0 0 0 0 1 1]
llr: [-1000. -1000.  1000.  1000.  1000.  1000. -1000. -1000.]
u_hat (natural order): [0 0 0 0 0 0 1 0]
expected info: [1 1 0 1]
decoded info : [0 0 1 0]
match? False


In [9]:
def monte_carlo_polar_matrix(
    N=128, K=None, p_design=0.05, p_channel=None,
    trials=1000, use_exact_f=True, debug=False
):
    if K is None:
        K = N//2
    if p_channel is None:
        p_channel = p_design

    frozen_mask, info_positions = select_frozen_direct(N, K, p_design)
    info_positions = np.sort(info_positions)

    # compute bit-reversal permutation & mapped frozen/u_frozen
    br = bit_reverse_indices(N)
    inv_br = np.empty_like(br); inv_br[br] = np.arange(N)
    frozen_mask_br = frozen_mask[inv_br]
    u_frozen_nat = np.zeros(N, dtype=np.int8)
    u_frozen_br = u_frozen_nat[inv_br]

    K_actual = len(info_positions)
    bit_err = 0
    block_err = 0

    for t in range(trials):
        msg = np.random.randint(0, 2, size=K_actual).astype(np.int8)
        u = np.zeros(N, dtype=np.int8)
        u[info_positions] = msg

        x = (G_matrix @ u) % 2
        y = bsc(x, p_channel)
        llr = bsc_llr(y, p_channel)

        # decode with decoder-order frozen mask
        uhat_br = sc_decode(llr, frozen_mask_br, u_frozen=u_frozen_br, use_exact_f=use_exact_f)
        uhat = uhat_br[inv_br]   # back to natural order

        msg_hat = uhat[info_positions]
        errs = int((msg_hat != msg).sum())
        bit_err += errs
        block_err += int(errs > 0)

        if debug and (t % max(1, trials//10) == 0):
            print(f"Trial {t}/{trials}: bit_err={bit_err}, block_err={block_err}")

    BER = bit_err / (trials * K_actual)
    BLER = block_err / trials
    return BER, BLER



K = 4
ber, bler = monte_carlo_polar_matrix(
    N=N,
    K=K,
    p_design=0.05,
    p_channel=0.0,   # <--- noiseless
    trials=200,
    debug=True
)
print("No-noise:", ber, bler)


Trial 0/200: bit_err=0, block_err=0
Trial 20/200: bit_err=46, block_err=18
Trial 40/200: bit_err=99, block_err=36
Trial 60/200: bit_err=141, block_err=55
Trial 80/200: bit_err=182, block_err=74
Trial 100/200: bit_err=232, block_err=92
Trial 120/200: bit_err=275, block_err=109
Trial 140/200: bit_err=319, block_err=127
Trial 160/200: bit_err=364, block_err=144
Trial 180/200: bit_err=408, block_err=161
No-noise: 0.5625 0.89
